In [151]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import chisquare
import numpy

#Reading normalised data
data = pd.read_csv('normalized_ferret_data.csv')
data.sort_values("Gene", inplace = True) 

#Getting rid of duplicated genes

data.drop_duplicates(subset ="Gene", 
                     keep = False, inplace = True)

#Computing the raw average gene expression values for each item

nctrl_1_avr = (data['Series10_FerretNW_Ctl_d1_1'] + data['Series10_FerretNW_Ctl_d1_2'])/2
ncov_1_avr = (data['Series10_FerretNW_SARS.CoV.2_d1_1'] + data['Series10_FerretNW_SARS.CoV.2_d1_2'])/2

nctrl_3_avr = (data['Series11_FerretNW_Ctl_d3_1'] + data['Series11_FerretNW_Ctl_d3_2'])/2
ncov_3_avr = (data['Series11_FerretNW_SARS.CoV.2_d3_1'] + data['Series11_FerretNW_SARS.CoV.2_d3_2'])/2

tctrl_3_avr = (data['Series14_FerretTrachea_Ctl_d3_1'] + data['Series14_FerretTrachea_Ctl_d3_2'] + data['Series14_FerretTrachea_Ctl_d3_3'] + data['Series14_FerretTrachea_Ctl_d3_4'])/4
tcov_3_avr = (data['Series14_FerretTrachea_SARS.CoV.2_d3_1'] + data['Series14_FerretTrachea_SARS.CoV.2_d3_2'] + data['Series14_FerretTrachea_SARS.CoV.2_d3_3'] + data['Series14_FerretTrachea_SARS.CoV.2_d3_4'])/4
tiav_3_avr = (data['Series14_FerretTrachea_IAV_d3_1'] + data['Series14_FerretTrachea_IAV_d3_2'] + data['Series14_FerretTrachea_IAV_d3_3'] + data['Series14_FerretTrachea_IAV_d3_4'] + data['Series14_FerretTrachea_IAV_d3_5'] + data['Series14_FerretTrachea_IAV_d3_6'])/6

nctrl_7_avr = (data['Series12_FerretNW_Ctl_d7_1'] + data['Series12_FerretNW_Ctl_d7_2'])/2
ncov_7_avr = (data['Series12_FerretNW_SARS.CoV.2_d7_1'] + data['Series12_FerretNW_SARS.CoV.2_d7_2'])/2
niav_7_avr = (data['Series12_FerretNW_IAV_d7_1'] + data['Series12_FerretNW_IAV_d7_2'])/2

nctrl_14_avr = (data['Series13_FerretNW_Ctl_d14_1'] + data['Series13_FerretNW_Ctl_d14_2'])/2
ncov_14_avr = (data['Series13_FerretNW_SARS.CoV.2_d14_1'] + data['Series13_FerretNW_SARS.CoV.2_d14_2'])/2

def clean_data(data,control,covid,bar):
    drops=[]
    
    for i in data.index:
        if abs(control[i]-covid[i]) <= bar or abs(control[i]-covid[i])/(covid[i]+0.1) <=0.5 or control[i]==0 or covid[i]==0:
                drops += [i];
    cleaned_ctrl = control.drop(drops, axis = 0)
    cleaned_cov = covid.drop(drops,axis=0)
    return cleaned_ctrl, cleaned_cov

day1_ctrl, day1_cov = clean_data(data, nctrl_1_avr,ncov_1_avr,1)
day3_ctrl, day3_cov = clean_data(data,nctrl_3_avr,ncov_3_avr,1)
day7_ctrl, day7_cov = clean_data(data,nctrl_7_avr,ncov_7_avr,1)
day14_ctrl, day14_cov = clean_data(data,nctrl_14_avr,ncov_14_avr,1)

#Returns a list of relevant genes sorted by chi-square in descending order for each day

def sig_genes(ctrl,cov, day):
    chisquares = []
    for i in range(len(ctrl)):
        chisquares.append(chisquare(ctrl.values[i],cov.values[i])[0])
                          
    chi_incl_comp_df = pd.DataFrame([ctrl,cov],index = ['control_day'+str(day), 'covid_day'+str(day)]).T
    chi_incl_comp_df['chi'] = chisquares
    sorted_by_chi = chi_incl_comp_df.sort_values(by = ['chi'],ascending = False)
    return sorted_by_chi

#Sorted list of genes for each day

sorted_day1 = sig_genes(day1_ctrl, day1_cov,1)
sorted_day3 = sig_genes(day3_ctrl, day3_cov,3)
sorted_day7 = sig_genes(day7_ctrl, day7_cov,7)
sorted_day14 = sig_genes(day14_ctrl, day14_cov,14)

#Writing data for each day into csv files
sorted_day1.to_csv('day1_comparison.csv')
sorted_day3.to_csv('day3_comparison.csv')
sorted_day7.to_csv('day7_comparison.csv')
sorted_day14.to_csv('day14_comparison.csv')

In [74]:
#Returns genes that have significant chi-square values in one of the 4 days

def get_genes(day1,day3,day7,day14):
    genes = []
    for i in sorted_day1.index:
        if i in sorted_day3.index[:1000] or i in sorted_day7.index[:1000] or i in sorted_day14.index[0:1000]:
            genes.append(data.loc[i])
    return genes

#Putting together an overall file with relevant genes and chi-square values

overall = pd.concat([sorted_day1,sorted_day3,sorted_day7,sorted_day14],axis=1)

#Return the set of relevant genes that have high significant values
important_genes = pd.DataFrame(get_genes(sorted_day1,sorted_day3,sorted_day7,sorted_day14))

important_genes.to_csv('important_genes.csv')
overall.to_csv('overall.csv')

In [75]:
#Computing the average across all days for control and covid group

ctrl_avr_alldays = (nctrl_1_avr+ nctrl_1_avr+ nctrl_1_avr + nctrl_1_avr)/4
cov_avr_alldays = (ncov_1_avr+ ncov_1_avr+ ncov_1_avr + ncov_1_avr)/4

#Produces list of significantly different genes in desceding order wrt chisquare values
sorted_by_chi_all = sig_genes(cov_avr_alldays, ctrl_avr_alldays,'all')

In [182]:
#Dropping the NaN and inf Values

drops = []
simp = sorted_by_chi_all
for i in sorted_by_chi_all.index:
    if ( simp['control_dayall'][i]== 0) or (simp['covid_dayall'][i] == 0):
        drops += [i];

newsorted_dayall = sorted_by_chi_all.drop(drops,axis=0)

# Look for an intersection between 1) relevant genes found with a significant difference between covid and control
# for each day and, 2) relevant genes found with significant difference between covid and control on average across 4 days.

counter = 0
genes_2 = []
for i in newsorted_dayall.index[:50]:
    if i in important_genes.index and data.loc[i].Human_Gene == data.loc[i].Human_Gene:
        counter += 1
        genes_2.append(data.loc[i])
        print(data.loc[i].Human_Gene)
print('counter: '+str(counter))

selected_genes = pd.DataFrame(genes_2)
selected_genes

SMARCB1
TBC1D14
TOX2
PDAP1
AMIGO2
IRS4
TDG
PIK3C2G
SYNRG
IFNA21
MIR144
SOX13
CAPNS1
TMEM220
counter: 14


,Gene,Series10_FerretNW_Ctl_d1_1,Series10_FerretNW_Ctl_d1_2,Series10_FerretNW_SARS.CoV.2_d1_1,Series10_FerretNW_SARS.CoV.2_d1_2,Series11_FerretNW_Ctl_d3_1,Series11_FerretNW_Ctl_d3_2,Series11_FerretNW_SARS.CoV.2_d3_1,Series11_FerretNW_SARS.CoV.2_d3_2,Series12_FerretNW_Ctl_d7_1,...,Series14_FerretTrachea_IAV_d3_2,Series14_FerretTrachea_IAV_d3_1,Series14_FerretTrachea_IAV_d3_4,Series14_FerretTrachea_IAV_d3_6,Series14_FerretTrachea_SARS.CoV.2_d3_1,Series14_FerretTrachea_SARS.CoV.2_d3_2,Series14_FerretTrachea_SARS.CoV.2_d3_3,Series14_FerretTrachea_SARS.CoV.2_d3_4,Human_ID,Human_Gene
3744,ENSMPUG00000004371,308242.213700,149510.566000,65827.801080,76008.283070,188765.870500,94541.841770,25295.954690,213117.549000,27320.558460,...,421.009080,348.525240,329.035238,427.614248,303.875531,458.218905,380.320909,473.955931,ENSG00000099956,SMARCB1
25656,ENSMPUG00000020575,47698.814940,24605.995590,1774.469092,7154.872553,24201.555070,4412.767480,2191.657188,23631.266830,2363.500326,...,5.062233,6.454171,5.483921,19.496090,0.520335,12.584364,2.690950,14.426591,ENSG00000132405,TBC1D14
6757,ENSMPUG00000017848,230.954103,15.752878,2555.077527,151.182730,8662.487643,625.037887,1031.674072,0.000000,116.988767,...,55209.561530,25704.197490,40974.758240,35885.803620,2314.969582,4210.210055,8961.311420,11366.031810,ENSG00000124191,TOX2
27469,ENSMPUG00000017726,5637.997209,7254.200363,5798.617469,23059.695360,18192.893120,13124.406650,4631.262935,10573.273750,8330.745526,...,63.277918,65.463736,121.743038,81.883579,42.667455,55.149125,27.806482,22.488509,ENSG00000106244,PDAP1
21000,ENSMPUG00000016737,4557.947141,7797.674658,8006.172860,19745.663360,9780.766395,20862.375680,4955.503358,12469.854730,14896.024210,...,113.056547,94.046493,131.614095,89.682016,43.708124,31.090782,55.612963,52.190313,ENSG00000139211,AMIGO2
23467,ENSMPUG00000013992,353.223921,94.517269,3160.608523,125.874608,7222.912384,29.168435,674.489435,6.608296,41.723266,...,1898.337537,1026.213208,676.715807,1213.956558,306.477205,760.983901,1351.305305,1236.019362,ENSG00000133124,IRS4
4116,ENSMPUG00000012618,2520.116825,2236.908690,3007.909402,10000.038290,3066.921203,6401.776933,3431.399981,5630.268270,5563.102187,...,294.453245,349.447265,398.132638,266.446568,240.394683,227.999068,200.924254,184.575497,ENSG00000139372,TDG
13463,ENSMPUG00000010447,2221.235045,3268.722205,3629.236859,9165.536264,2768.574447,7157.378289,2807.193820,8319.844780,10544.532960,...,158.616648,205.611451,156.840130,157.268462,269.533433,104.006068,288.828615,285.561631,ENSG00000139144,PIK3C2G
7221,ENSMPUG00000005377,5692.339351,3197.834254,3494.966942,14680.708900,5593.480099,10143.670410,4152.704879,5775.650785,8645.715282,...,686.776336,724.711214,704.135410,714.856646,372.559726,484.868147,263.264591,210.458498,ENSG00000275066,SYNRG
17980,ENSMPUG00000004686,1392.517383,1953.356884,2393.163804,6023.999092,2357.565279,3841.899543,2408.395439,4857.097628,5205.591062,...,38.810456,47.945271,27.419603,59.788010,64.521517,61.071179,73.552629,70.860018,ENSG00000137080,IFNA21


In [184]:
#Plotting the ratio
fig_diff = px.scatter()
for i in selected_genes.index:
    fig_diff.add_trace(go.Scatter(x = [1,3,7,14], y = [abs(ncov_1_avr[i]-nctrl_1_avr[i])/nctrl_1_avr[i],
                                                  abs(ncov_3_avr[i]-nctrl_3_avr[i])/nctrl_3_avr[i],
                                                  abs(ncov_7_avr[i]-nctrl_7_avr[i])/nctrl_7_avr[i],
                                                  abs(ncov_14_avr[i]-nctrl_14_avr[i])/nctrl_14_avr[i]], 
                                                  name = (selected_genes.Gene[i])[-5:]))
fig_diff.update_layout(
    title={
        'x':0.5,
        'text': " Ratio of normalised RNA expression of Covid-normal with time evolution ",
        },
    xaxis_title="Days",
    yaxis_title="Ratio",
    font=dict(
        size=13,
        color="#7f7f7f"
    )
)

fig_diff.update_xaxes(tickvals=[1, 3, 7, 14])
fig_diff.write_html('diff_evolve.html')

#Plotting Selected Covid group time evolution
fig_cov = px.scatter()
for i in selected_genes.index:
    fig_cov.add_trace(go.Scatter(x = [1,3,7,14], y = [ncov_1_avr[i],
                                                  ncov_3_avr[i],
                                                  ncov_7_avr[i],
                                                  ncov_14_avr[i]], 
                                                  name = selected_genes.Gene[i][-5:]))

fig_cov.update_layout(
    title={
        'x':0.5,
        'text': " Normalised RNA expression of Covid group ferrets nasal cell with time evolution ",
        },
    xaxis_title="Days",
    yaxis_title="Gene Count Normalised Value",
    font=dict(
        size=13,
        color="#7f7f7f"
    )
)

fig_cov.update_xaxes(tickvals=[1, 3, 7, 14])
fig_cov.write_html('cov_evolve.html')

#Plotting Selected Control group time evolution

fig_ctrl = px.scatter()
for i in selected_genes.index:
    fig_ctrl.add_trace(go.Scatter(x = [1,3,7,14], y = [nctrl_1_avr[i],
                                                  nctrl_3_avr[i],
                                                  nctrl_7_avr[i],
                                                  nctrl_14_avr[i]], 
                                                  name = (selected_genes.Gene[i])[-5:]))
    
fig_ctrl.update_layout(
    title={
        'x':0.5,
        'text': " Normalised RNA Expression of Control group with time evolution ",
        },
    xaxis_title="Days",
    yaxis_title="Gene Count Normalised Value",
    font=dict(
        size=13,
        color="#7f7f7f"
    )
)

fig_ctrl.update_xaxes(tickvals=[1, 3, 7, 14])
fig_ctrl.write_html('ctrl_evolve.html')

In [191]:
#Getting a list of RNA names from a list of indices:

def get_genenames(sorted_data, num):
    x_genenames = []
    for i in sorted_data.index[:num]:
        x_genenames.append(data.loc[i].Gene)
    return x_genenames

# Chi-square Plot for 10 genes with highest chi-square values between 
# control and covid group averaing across the days

x_genenames = get_genenames(newsorted_dayall,10)
fig_chi = px.bar(x = x_genenames, y = newsorted_dayall['chi'][:10] )
fig_chi.update_layout(
    title={
            'x':0.5,
            'text': " highest 10 Chi-squared values for relevant RNAs on average ",
            },
        xaxis_title="RNA Name",
        yaxis_title="Chi-squared value",
        font=dict(
            size=13,
            color="#7f7f7f"
    ))
fig_chi.write_html('chisquare_alldays.html')

In [186]:
# A combination of bar plots of chi-square sgnificance for each day (4 subplots)
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig_4plots = make_subplots(rows=2, cols=2, subplot_titles=("Day 1", "Day 3", "Day 7", "Day 14"))

genenames_1 = get_genenames(sorted_day1,10)
genenames_3 = get_genenames(sorted_day3,10)
genenames_7 = get_genenames(sorted_day7,10)
genenames_14 = get_genenames(sorted_day14,10)

fig_4plots.add_trace(
    go.Bar(x=genenames_1, y=sorted_day1['chi'][:10]),
    row=1, col=1
)

fig_4plots.add_trace(
    go.Bar(x=genenames_3, y=sorted_day3['chi'][:10]),
    row=1, col=2
)

fig_4plots.add_trace(
    go.Bar(x=genenames_7, y=sorted_day7['chi'][:10]),
    row=2, col=1
)

fig_4plots.add_trace(
    go.Bar(x=genenames_14, y=sorted_day14['chi'][:10]),
    row=2, col=2
)

fig_4plots.update_layout(height=1100, width=1000, 
                         title_text="highest 10 Chi-squared values for relevant RNAs each day", 
                         title_x = 0.5,
                         showlegend = False
                        )

# Update xaxis properties
fig_4plots.update_xaxes(title_text="RNA Names", row=1, col=1)
fig_4plots.update_xaxes(title_text="RNA Names", row=1, col=2)
fig_4plots.update_xaxes(title_text="RNA Names", row=2, col=1)
fig_4plots.update_xaxes(title_text="RNA Names", row=2, col=2)

# Update yaxis properties
fig_4plots.update_yaxes(title_text="Chi-square Value", row=1, col=1)
fig_4plots.update_yaxes(title_text="Chi-square Value", row=2, col=1)

fig_4plots.write_html('1_plot_each_day.html')

In [187]:
genenames_all_long = get_genenames(newsorted_dayall,len(newsorted_dayall))

In [188]:
newsorted_dayall['Gene'] = genenames_all_long
newsorted_dayall['index'] = range(1,len(newsorted_dayall)+1)
newsorted_dayall =newsorted_dayall[['index','Gene','control_dayall','covid_dayall','chi']]

In [173]:
newsorted_dayall

,index,Gene,control_dayall,covid_dayall,chi
3744,1,ENSMPUG00000003746,70918.042075,228876.389850,109014.475666
6215,2,ENSMPUG00000006217,260.213352,28302.754300,27784.719977
25656,3,ENSMPUG00000025669,4464.670823,36152.405265,27774.431791
7576,4,ENSMPUG00000007578,67011.654795,116498.861580,21021.524178
29769,5,ENSMPUG00000029817,11240.436908,3195.138254,20257.912273
2446,6,ENSMPUG00000002448,28896.218150,13337.520295,18149.781488
6757,7,ENSMPUG00000006759,1353.130129,123.353490,12260.298240
8830,8,ENSMPUG00000008832,14690.324750,6113.979990,12030.410559
27469,9,ENSMPUG00000027500,14429.156414,6446.098786,9886.477266
21000,10,ENSMPUG00000021002,13875.918110,6177.810900,9592.532952


NameError: name 'NaN' is not defined